# Hadamard Test

Consider the observable $O$ and two generic quantum states $| \psi\rangle$ and $| \phi\rangle$. We want to calculate the quantity
$$
\langle \psi | O | \psi\rangle.
$$
where $O$ is a Pauli operator.

First of all we shall prepare the states $|\psi\rangle$ and $|\phi\rangle$ using a quantum circuit for each of them. So we  have
$$
| \psi\rangle = U_{\psi}|0\rangle \qquad |\phi\rangle = U_{\phi}|0\rangle
$$

Let's define an observable we want to use:
$$
O = X_1X_2
$$

Now we can evaluate the matrix element using the following fact:
$$
\langle \psi|O|\phi\rangle = \langle 0 |U_\psi^\dagger O U_\phi |0\rangle
$$
This is just an expectation value which can be solved with a simple Hadamard test. The probability to measure $0$ or $1$ in the ancilla qubit is

$$
P(0) = \frac{1}{2} \left[ I + \operatorname{Re} \langle \psi| O |\phi\rangle \right]
$$

$$
P(1) = \frac{1}{2} \left[ I - \operatorname{Re} \langle \psi| O | \phi\rangle \right]
$$

The difference between the probability of $0$ and $1$ gives

$$
\langle X\rangle = P(0)-P(1) = \operatorname{Re} \langle \psi | O |\phi\rangle.
$$

Similarly, the imaginary part can be obtained from Y measurement
$$
\langle Y\rangle = \operatorname{Im} \langle \psi | O | \phi\rangle.
$$

Combining these results, the quantity $\langle \psi | O | \psi\rangle$ is obtained.

### Numerical result as a reference:

In [ ]:
import numpy as np

In [ ]:
import cudaq

In [ ]:
num_qubits = 2

In [ ]:
@cudaq.kernel
def psi(num_qubits: int):
    q = cudaq.qvector(num_qubits)
    h(q[1])

In [ ]:
@cudaq.kernel
def phi(num_qubits: int):
    q = cudaq.qvector(num_qubits)
    x(q[0])

In [ ]:
psi_state = cudaq.get_state(psi, num_qubits)
print("Psi state: ", np.array(psi_state))

In [ ]:
phi_state = cudaq.get_state(phi, num_qubits)
print("Phi state: ", np.array(phi_state))

In [ ]:
ham = cudaq.spin.x(0) * cudaq.spin.x(1)
ham_matrix = ham.to_matrix()
print("hamiltonian: ", np.array(ham_matrix), "\n")

In [ ]:
num_ev = np.array(psi_state).conj() @ ham_matrix @ np.array(phi_state).T

In [1]:
print("Numerical expectation value: ", num_ev)

Psi state:  [0.70710677+0.j 0.        +0.j 0.70710677+0.j 0.        +0.j]
Phi state:  [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
hamiltonian:  [[0.+0.j 0.+0.j 0.+0.j 1.+0.j]
 [0.+0.j 0.+0.j 1.+0.j 0.+0.j]
 [0.+0.j 1.+0.j 0.+0.j 0.+0.j]
 [1.+0.j 0.+0.j 0.+0.j 0.+0.j]] 

Numerical expectation value:  (0.7071067690849304+0j)


### Using ``observe`` algorithmic primitive to compute the expectation value for ancilla qubits.

In [ ]:
import cudaq

In [ ]:
@cudaq.kernel
def u_psi(q: cudaq.qview):
    h(q[1])

In [ ]:
@cudaq.kernel
def u_phi(q: cudaq.qview):
    x(q[0])

In [ ]:
@cudaq.kernel
def apply_pauli(q: cudaq.qview):
    x(q[0])
    x(q[1])

In [ ]:
@cudaq.kernel
def kernel(num_qubits: int):
    ancilla = cudaq.qubit()
    q = cudaq.qvector(num_qubits)
    h(ancilla)
    cudaq.control(u_phi, ancilla, q)
    cudaq.control(apply_pauli, ancilla, q)
    cudaq.control(u_psi, ancilla, q)

In [ ]:
num_qubits = 2
shots = 100000
x_0 = cudaq.spin.x(0)
y_0 = cudaq.spin.y(0)
results = cudaq.observe(kernel, [x_0, y_0], num_qubits, shots_count=shots)
evs = np.array([result.expectation() for result in results])
std_errs = np.sqrt((1 - evs**2) / shots)

In [2]:
print(f"QC result: {evs[0]}+{evs[1]}i ± {std_errs[0]}+{std_errs[1]}i")
print("Numerical result", num_ev)

QC result: 0.70652+0.0046399999999999775i ± 0.0022379220040028205+0.003162243618698597i
Numerical result (0.7071067690849304+0j)


In [3]:
print(cudaq.__version__)

CUDA-Q Version cu12-0.9.0 (https://github.com/NVIDIA/cuda-quantum 77a1c80a18896b4c7ff4ece99f06e6a62c8a28ef)
